In [30]:
import cv2
import os
import numpy as np

In [31]:
def image_segmentation_algorithm(input_img):
    
    # Creating kernel
    kernel = np.ones((2, 2), np.uint8)

    # Using cv2.erode() method 
    img = cv2.resize(input_img, dsize=NEW_IMG_SIZE)
    # img = cv2.erode(img, kernel, iterations = 5) 
    # img = cv2.dilate(img, kernel, iterations = 5)

    twoDimage = img.reshape((-1,3))
    twoDimage = np.float32(twoDimage)

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 5
    attempts=10

    ret,label,center=cv2.kmeans(twoDimage,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    result_image = res.reshape((img.shape))
    
    return result_image

In [32]:
def reject_outliers(data, m=0.7):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [33]:
def identify_center(input_img):

    input_img = cv2.cvtColor(input_img, cv2.COLOR_RGB2HSV)

    min_diff = np.min(np.abs(input_img[:,:,0] - 240 * np.ones(input_img.shape[:-1])))
    blue_color = 240 + (min_diff if (240 + min_diff) in input_img else -min_diff)

    h_range = 10

    x_blues, y_blues = [], []

    for i in range(input_img.shape[0]):
        for j in range(input_img.shape[1]):
            if input_img[i][j][0] >= (blue_color - h_range) and input_img[i][j][0] <= (blue_color + h_range):
                x_blues.append(i)
                y_blues.append(j)
    
    center_index = (int(np.median(reject_outliers(np.array(x_blues)))), int(np.median(reject_outliers(np.array(y_blues)))))

    # is_blue = (input_img >= (blue_color - h_range) and input_img <= (blue_color + h_range))
    # is_blue = is_blue[:, :, 0]

    # print(np.where(is_blue))

    # center_index = np.int32(np.mean(np.where(is_blue), axis=-1))

    result_img = input_img

    for deltax in range(-3, 3):
        for deltay in range(-3, 3):
            try:
                red = np.array([180, 67, 80])
                result_img[center_index[0] + deltax, center_index[1] + deltay, :] = np.array([0, 255, 0])
            except:
                print("On Edge")

    return cv2.cvtColor(result_img, cv2.COLOR_HSV2RGB)


In [37]:
vidcap = cv2.VideoCapture('Stable.mov')

success, image = vidcap.read()
NEW_IMG_SIZE = (image.shape[1] // 10, image.shape[0] // 10)
count = 1

In [35]:
%%time
while success:
  cv2.imwrite(f"processed/frame{count}.jpg", image_segmentation_algorithm(image))     # save frame as JPEG file      
  success, image = vidcap.read()
  count += 1

CPU times: user 2min 37s, sys: 13.6 s, total: 2min 50s
Wall time: 46.7 s


In [38]:
%%time
for i in range(1, 334):
    cv2.imwrite(f"centered/frame{i}.jpg", identify_center(cv2.imread(f"processed/frame{i}.jpg")))

CPU times: user 5.72 s, sys: 195 ms, total: 5.91 s
Wall time: 6.06 s


In [ ]:
os.system("ffmpeg -framerate 30 -i processed/frame%d.jpg -c:v libx264 -r 30 new_output_3.mp4")